<a href="https://colab.research.google.com/github/MasahiroAraki/MachineLearning3/blob/master/notebook/chap13ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers[ja] --quiet

# 系列ラベリング

## 例題13.1

Transformers ライブラリを用いて，BERT を用いた固有表現抽出プログラムを作成せよ．

In [ ]:
from transformers import pipeline
from pprint import pprint

ner_pipeline = pipeline("token-classification",
    model="llm-book/bert-base-japanese-v3-ner-wikipedia-dataset",
    aggregation_strategy="simple",  # 複数トークンをまとめる方法
)

In [ ]:
text = "サムは浅葱町でプリウスを見た"
pprint(ner_pipeline(text))

# 系列識別

## 例題13.2

Transformers ライブラリを用いて，BERT を用いた自然言語文のPN判定を行うプログラムを作成せよ．

In [ ]:
clf_pipeline = pipeline("text-classification",
                        model="llm-book/bert-base-japanese-v3-marc_ja")

In [ ]:
text = "その映画は二度と観ることはないだろう"
print(clf_pipeline(text)[0])

# 系列変換

## 例題13.3

Transformers ライブラリを用いて，GPT を用いた対話的な応答生成を行うプログラムを作成せよ．

In [ ]:
gen_pipeline = pipeline("text-generation",
                        model="rinna/japanese-gpt2-xsmall")

In [ ]:
text = "曇の日の遊びのおすすめは？"
print(gen_pipeline(text)[0])

# 時系列信号の予測

時系列機械学習用ライブラリ[sktime](https://github.com/sktime/sktime) (バージョン 0.39.0)（dask==2025.9.1）を使って、時系列信号の予測を行います。

エラーが出る場合は、以下のライブラリのインストールのあと、ランタイムを再起動します

In [ ]:
!pip install -U sktime[forecasting]==0.39.0 dask[dataframe]==2025.9.1 --quiet


データセットとして、Airline Passengers を使います。これは、1949年から1960年までの月ごとの飛行機の乗客数データです。

In [ ]:
import numpy as np
import pandas as pd
import tqdm
from sktime.datasets import load_airline
from sktime.utils.plotting import plot_series
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.statsforecast import StatsForecastAutoARIMA

In [ ]:
y = load_airline()
y

時系列の線グラフで表示してわかるように、季節変動を伴いながら全体として乗客数は増えています。このデータから直近36ヶ月のデータをテストデータ、それ以前のデータを学習データとして予測問題を設定します。

In [ ]:
fig, ax = plot_series(y)
fig.savefig('s.svg', format='svg', bbox_inches='tight')

In [ ]:
y_train, y_test = temporal_train_test_split(y,test_size=36)

sktimeのplot_seriesでは複数の系列データを異なる色で表示する機能もあります。

In [ ]:
fig, ax = plot_series(y_train, y_test, labels=["y_train", "y_test"])
fig.savefig('s.svg', format='svg', bbox_inches='tight')

周期を表すパラメータspを1年の月数である12にしてAutoARIMAのインスタンスを作成し、学習させます

In [ ]:
forecaster = StatsForecastAutoARIMA(sp=12)
forecaster.fit(y_train)

テストデータと同じ系列長だけ予測をさせます。

In [ ]:
y_pred = forecaster.predict(list(range(1, len(y_test)+1)))
fig, ax = plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])
fig.savefig('s.svg', format='svg', bbox_inches='tight')

## 演習問題

13.1 Transformers ライブラリを用いて，日英の翻訳を行うプログラムを作成せよ．

In [ ]:
from transformers import pipeline
from pprint import pprint

translate_pipeline = pipeline("translation", model="Helsinki-NLP/opus-mt-ja-en")
translate_pipeline("長い文の翻訳は苦手です。")[0]['translation_text']